In [3]:
# based on https://math.libretexts.org/Bookshelves/Scientific_Computing_Simulations_and_Modeling/Introduction_to_the_Modeling_and_Analysis_of_Complex_Systems_(Sayama)/16%3A_Dynamical_Networks_I__Modeling/16.02%3A_Simulating_Dynamics_on_Networks

In [2]:
pip install networkx

  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\124109249\Downloads\JupyterLab-Portable-69696969\apps\python.exe -m pip install --upgrade pip' command.


In [6]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import pycxsimulator
# pycxsimulator can be downloaded from https://github.com/hsayama/PyCX/blob/master/pycxsimulator.py

# Parameters
N = 30  # Number of nodes
p = 0.5  # Probability of edge creation
epsilon = 0.4  # Confidence threshold for interaction
tau = 0.15 # threshold opinion difference to increase or decrease weight
t = 0

def initialize():
    global g, pos
    g = nx.erdos_renyi_graph(N, p)  # create erdos renyi graph with N nodes and probability of edge forming being 0.5
    pos = nx.spring_layout(g)  # layout positions of nodes

    #  initialise opinion value randomly from [0,1]
    for node in g.nodes:
        g.nodes[node]['opinion'] = random.random()
        
    # initialise weight (perhaps relationship strength) randomly from [0,1]
    for node1, node2 in g.edges:
        
        g[node1][node2]['weight'] = random.random() 

def observe():
    global g
    plt.cla()
    
    # get node states (opinion value)
    opinions = [g.nodes[node]['opinion'] for node in g.nodes]
    
    # get weight between nodes
    edge_weights = [g[u][v]['weight'] for u, v in g.edges]
    
    # get minimum weight and maximum weight to perform min-max normalisation for colormapping values
    min_weight = min(edge_weights)
    max_weight = max(edge_weights)
    edge_colors = [plt.cm.viridis((weight - min_weight) / (max_weight - min_weight)) for weight in edge_weights]
    
    # draw the network with node colours representing intensity of opinions, 
    # weight colour representing strength of connection between nodes
    nx.draw(g, pos=pos, node_color=opinions, cmap='coolwarm', vmin=0, vmax=1, with_labels=True, width=edge_weights, edge_color=edge_colors)
    plt.draw()

def update():
    global g
    
    # select a random edge connecting 2 nodes
    if g.edges:
        node1, node2 = random.choice(list(g.edges))

        # fetch the states of the 2 nodes
        opinion1 = g.nodes[node1]['opinion']
        opinion2 = g.nodes[node2]['opinion']

        # update changes in node states only when the absolute difference of their states is smaller than epsilon
        if abs(opinion1 - opinion2) < epsilon:
            new_opinion1 = opinion1 + g[node1][node2]['weight'] * (opinion2 - opinion1)
            new_opinion2 = opinion2 + g[node1][node2]['weight'] * (opinion1 - opinion2)

            # update the new states of the nodes
            g.nodes[node1]['opinion'] = new_opinion1
            g.nodes[node2]['opinion'] = new_opinion2
            
            # update weight between the two nodes
            # For example, use the opinion difference to adjust the weight
            weight_change = abs(opinion1 - opinion2)  
            if weight_change <= tau:
                g[node1][node2]['weight'] = max(0, g[node1][node2]['weight'] + weight_change)  # ensure positive weight
            elif weight_change > tau:
                g[node1][node2]['weight'] = max(0, g[node1][node2]['weight'] - weight_change)

# Start the simulation
pycxsimulator.GUI().start(func=[initialize, observe, update])

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\124109249\Downloads\JupyterLab-Portable-69696969\apps\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\124109249\Downloads\JupyterLab-Portable-69696969\apps\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\124109249\Downloads\JupyterLab-Portable-69696969\notebooks\pycxsimulator.py", line 240, in stepModel
    self.modelStepFunc()
  File "C:\Users\124109~1\AppData\Local\Temp/ipykernel_7404/850682310.py", line 52, in update
    t+=1
UnboundLocalError: local variable 't' referenced before assignment
